In [54]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [55]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [56]:
from implementations import *

## Do your thing crazy machine learning thing here :) ...

### Testing functions

In [68]:
#GD
initial_w=np.zeros(tX.shape[1])
max_iters=20
gamma=0.01
w,loss = least_squares_GD(y,tX,initial_w,max_iters,gamma)
print(f'w:{w}\nloss:{loss}')

w:[ 7.78069080e+89  3.75595575e+89  5.33146999e+89  2.10447516e+89
 -7.26881329e+90 -7.34784021e+90 -7.26843738e+90  1.82589402e+88
  9.76942216e+88  7.20271270e+89  9.45313458e+87 -3.51706697e+87
 -7.26859520e+90  2.34216643e+89 -1.17786028e+86 -8.96959680e+85
  2.94865547e+89 -2.17205354e+86  3.11643298e+86  2.31457642e+89
 -1.25424077e+86  1.08362944e+90  2.52915489e+87 -4.33607534e+90
 -4.50259146e+90 -4.50262992e+90 -7.27548994e+90 -7.26853993e+90
 -7.26854228e+90  1.59083782e+89]
loss:3.782829607510557e+179


In [59]:
#SDG
initial_w=np.zeros(tX.shape[1])
max_iters=20
gamma=0.0001
w,loss = least_squares_SGD(y,tX,initial_w,max_iters,gamma)
print(f'w:{w}\nloss:{loss}')

w:[-1.14177880e+37 -9.23466643e+36 -9.19520989e+36 -1.40024925e+37
 -1.14517606e+35 -1.41669410e+37 -2.12703926e+35 -1.70854133e+35
 -5.90760237e+36 -3.35953402e+37 -3.52723040e+35  1.09026157e+34
  8.56548786e+33 -3.67921963e+36 -1.10549306e+35 -9.95920841e+34
 -1.01297199e+37  7.80998009e+34 -6.43789692e+34 -3.61752551e+36
 -3.33732344e+35 -4.97169147e+37 -2.58385384e+35 -1.05163270e+37
 -9.67934829e+34  4.35085292e+35 -9.25236128e+36 -2.62605373e+35
  2.56673692e+35 -1.97862644e+37]
loss:9.292159809852051e+77


In [60]:
#LS
w,loss = least_squares(y,tX)
print(f'w:{w}\nloss:{loss}')

w:[ 8.66449927e-03 -8.67853194e-03  4.09297842e-03 -8.80367749e-04
  2.98488106e-02 -4.71769577e-05 -4.00640280e-02 -9.86416872e-02
 -1.15656502e-03 -5.46401937e-05 -1.40907270e-01  7.92360038e-02
  2.59045315e-02  4.94027838e-03 -1.49793598e-03 -1.28588312e-03
 -1.08822300e-03 -2.52807414e-04  1.17754335e-03 -1.31328474e-03
  3.62511280e-05  1.82765835e-05 -3.66680254e-01  1.75936379e-04
 -1.72510726e-04  3.13320130e-04 -3.80980841e-04 -5.83914113e-03
 -8.98019819e-03  5.54269426e-04]
loss:0.3155719238476981


In [61]:
#RR
lambda_=0.1
w,loss = ridge_regression(y,tX,lambda_)
print(f'w:{w}\nloss:{loss}')

w:[ 8.66449749e-03 -8.67853397e-03  4.09297851e-03 -8.80368432e-04
  2.98486078e-02 -4.71770541e-05 -4.00640997e-02 -9.86418046e-02
 -1.15657194e-03 -5.46412338e-05 -1.40907154e-01  7.92359838e-02
  2.59047803e-02  4.94027345e-03 -1.49793599e-03 -1.28588239e-03
 -1.08823292e-03 -2.52808999e-04  1.17754341e-03 -1.31328786e-03
  3.62514153e-05  1.82757996e-05 -3.66678851e-01  1.75935821e-04
 -1.72510098e-04  3.13318989e-04 -3.80995170e-04 -5.83912382e-03
 -8.98017563e-03  5.54267225e-04]
loss:0.3155719238478015


In [70]:
#LR
initial_w=np.zeros(tX.shape[1])
max_iters=5
gamma=0.01
w,loss = logistic_regression(y_,tX,initial_w,max_iters,gamma)
print(f'w:{w}\nloss:{loss}')

w:[-2.70211090e+04 -9.03689665e+04 -2.20871046e+04 -2.83350241e+04
  1.14297779e+03 -2.71029888e+05  2.52144265e+02 -1.13847229e+03
 -1.49726885e+04 -9.31188272e+04 -1.60865871e+03  1.15689378e+03
  1.64801299e+03 -6.86754249e+03 -1.47372800e+01 -4.70078700e+01
 -3.26725701e+04  1.06899500e+00  1.82857150e+01 -3.92332897e+04
  7.41035800e+01 -1.27662515e+05 -1.54813000e+03  1.78834599e+05
  2.23868475e+05  2.23865161e+05 -4.09994204e+04  1.75428290e+03
  1.72423929e+03 -4.67402179e+04]
loss:nan


In [ ]:
#RLR
initial_w=np.zeros(tX.shape[1])
max_iters=5
gamma=0.00001
lambda_=0.01
w,loss = reg_logistic_regression(y,tX, lambda_, initial_w, max_iters,gamma)
print(f'w:{w}\nloss:{loss}')

### Cleaning dataset

In [45]:
tX[:,:][tX[:,:] == -999] = 0
#we could normalize the data ranging from [0,1] since its binary prediction",
y = (y-min(y))/(max(y)-min(y))


In [51]:
q1 = []
q2 = []
k = 1
for i in range(0,tX.shape[1]):
    q1 = np.percentile(tX[:,i],25)
    q2 = np.percentile(tX[:,i],50)
    q3 = np.percentile(tX[:,i],75)
    tX[:,i][(tX[:,i] < q1 - k*(q3-q1))] = q2
    tX[:,i][(tX[:,i] > q3 + k*(q3-q1))] = q2
    
#tX[:,:][tX[:,:] = -999].shape[0]

## Generate predictions and save ouput in csv format for submission:

In [47]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [11]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

NameError: name 'weights' is not defined